In [141]:
# ONLY USE THIS BLOCK OF CODE ON NICASIA'S COMPUTER 
import sys
sys.path.append("/anaconda/lib/python2.7/site-packages")
#####

import copy
import pandas as pd
import numpy as np
from matplotlib.patches import Polygon
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler as Standardize
from sklearn.linear_model import LinearRegression as Lin_Reg
from sklearn.linear_model import Ridge as Ridge_Reg
from sklearn.linear_model import Lasso as Lasso_Reg
from sklearn.preprocessing import PolynomialFeatures
from sklearn import ensemble
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from statsmodels.regression.linear_model import OLS
import string
import matplotlib.pyplot as plt
%matplotlib inline
import math

In [45]:
df = pd.read_csv("mn_crimes_and_schools.csv")
df = df[np.isfinite(df["zestimate_amount"])]

In [145]:
df_pct_with_reductions = pd.read_csv("new_predictors/Neighborhood_PctOfListingsWithPriceReductions_AllHomes.csv")

In [146]:
df_neighborhood_turnovers = pd.read_csv("new_predictors/Neighborhood_Turnover_AllHomes.csv")

In [55]:
df_subway_stops = pd.read_csv("/new_predictors/StationEntrances.csv")
df_subway_stops = df_subway_stops[["Latitude", "Longitude"]]

In [59]:
df_pct_with_reductions = df_pct_with_reductions[df_pct_with_reductions["County"]=="New York"] 

In [108]:
set(df["neighborhood"].unique()).difference(set(df_neighborhood_turnovers["RegionName"].unique()))

{'Astoria-Long Island City',
 'Bedford-Stuyvesant',
 'Fort Green',
 'North Sutton Area',
 'Soho',
 'Sunny Side',
 'Yorkville'}

In [147]:
df_neighborhood_turnovers = df_neighborhood_turnovers[df_neighborhood_turnovers["City"] == "New York"]

rename_dict = {'Astoria': 'Astoria-Long Island City', 'Bedford Stuyvesant': 'Bedford-Stuyvesant', \
              'Fort Greene': 'Fort Green', 'Sutton Place': 'North Sutton Area', 'SoHo': 'Soho', \
              'Sunnyside':'Sunny Side'}

df_neighborhood_turnovers["neighborhood"] = None
for i in df_neighborhood_turnovers.index:
    reg_name = df_neighborhood_turnovers.loc[i]["RegionName"]
    if reg_name in rename_dict.keys():
        df_neighborhood_turnovers.set_value(i, 'neighborhood', rename_dict[reg_name])
    else:
        df_neighborhood_turnovers.set_value(i, 'neighborhood', reg_name)

        
# this dataset is missing yorkville, so we're just making a copy of upper east side's data because yorkville seems to 
# just be a subset of the upper east side
yorkville_row = copy.deepcopy(df_neighborhood_turnovers.loc[6])
yorkville_row.neighborhood = "Yorkville"

df_neighborhood_turnovers = df_neighborhood_turnovers.append(yorkville_row)

# we just want the most recent rate and neighborhood name
df_neighborhood_turnovers = df_neighborhood_turnovers[["neighborhood", "2016-10"]]

## MERGE DF & DF_NEIGHBORHOOD_TURNOVERS SOMEHOW?? MATCH ON NEIGHBORHOOD 

In [ ]:
def manhattan_dist(lat1, lat2, long1, long2):
    return abs(long1-long2) + abs(lat1-lat2)

In [152]:
df["dist_to_subway"] = None
for i in df.index:
    lat = df.loc[i]["latitude"]
    lon = df.loc[i]["longitude"]
    mn_dists = [manhattan_dist(x[0], lat, x[1], lon) for x in df_subway_stops.as_matrix()]
    df.set_value(i, 'dist_to_subway', np.min(mn_dists))